In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import re
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers.merge import concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint

from sklearn.preprocessing import StandardScaler

Using TensorFlow backend.


In [3]:
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [4]:
df_data = pd.read_csv('../data/processed/featured_data.csv')

In [5]:
df_data.head()

,id,is_duplicate,common_word_ratio,fuzz_ratio,Partial_Ratio,Token_Sort_Ratio,Token_Set_Ratio,cleanQ1,cleanQ2
0,124269,0,0.125,28,38,43,46,legal ethic issu relat boyt good untri,polit divis issu untri would bizarr peopl untr...
1,227472,0,0.000,17,26,21,21,remov democraci india,biggest blunder made vajpaye govern
2,15911,0,0.600,88,89,87,88,watch fox news addict,watch fox news bad
3,326085,1,0.400,71,76,70,79,arnab goswami quit time,arnab goswami resign
4,237728,1,1.000,100,100,100,100,best friend date crush,best friend date crush


In [6]:
df=df_data[['cleanQ1','cleanQ2','is_duplicate']]
df['cleanQ1'] = df['cleanQ1'].apply(lambda x: str(x))
df['cleanQ2'] = df['cleanQ2'].apply(lambda x: str(x))

<ipython-input-6-57abb631de09>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cleanQ1'] = df['cleanQ1'].apply(lambda x: str(x))
<ipython-input-6-57abb631de09>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cleanQ2'] = df['cleanQ2'].apply(lambda x: str(x))


In [7]:
# merging questions of both Q1 and Q2 to a single list in which first 404287 index will be of question 1 and then rest of question 2
questions = list(df['cleanQ1']) + list(df['cleanQ2'])  # len(questions): 808574  [404287 of q1 and 404287 of q2]

tfidf = TfidfVectorizer() #  Convert a collection of raw documents to a matrix of TF-IDF features

tfidf.fit_transform(questions)  # Converting out text to a matrix of TF-IDF features

# mapping our feature_names with threre resptive tf-idf score  ( dict key:word and value:tf-idf score )
word2tfidf = dict(zip(tfidf.get_feature_names(), tfidf.idf_))

{'aa': 9.62547529713944, 'aaa': 10.852704963629643, 'aaac': 12.587306019017749, 'aaadhar': 12.992771127125913, 'aaai': 12.992771127125913, 'aabb': 12.992771127125913, 'aac': 12.992771127125913, 'aachar': 12.587306019017749, 'aachen': 11.48869373034964, 'aad': 12.992771127125913, 'aadaab': 12.587306019017749, 'aadhaar': 9.754092674961534, 'aadhar': 9.642867039851309, 'aadmi': 10.594875854327544, 'aadmk': 12.992771127125913, 'aadvantag': 12.587306019017749, 'aae': 12.587306019017749, 'aaft': 12.992771127125913, 'aag': 12.992771127125913, 'aah': 12.587306019017749, 'aahat': 12.992771127125913, 'aai': 11.606476766006024, 'aaj': 12.587306019017749, 'aakash': 10.594875854327544, 'aaksh': 12.992771127125913, 'aalavandhan': 12.992771127125913, 'aalto': 12.992771127125913, 'aam': 10.550424091756708, 'aamdaar': 12.992771127125913, 'aamir': 10.220182404886133, 'aammir': 12.992771127125913, 'aan': 12.992771127125913, 'aanandha': 12.992771127125913, 'aanika': 12.587306019017749, 'aankho': 12.587306

In [10]:
# Plotting Key Value Pair
wrd=list(word2tfidf.keys())[::-1][:10]   # finding keys of dict and then reversing them to get last 10 keys
for i in wrd:
    print(i,word2tfidf[i])

zzzz 12.992771127125913
zzzquil 11.894158838457804
zzz 12.992771127125913
zyzz 12.587306019017749
zyrtec 12.29962394656597
zynga 11.740008158630546
zylber 12.29962394656597
zyl 12.587306019017749
zygot 12.29962394656597
zycus 12.992771127125913


In [11]:
# en_vectors_web_lg, which includes over 1 million unique vectors.
nlp = spacy.load('en_core_web_sm')

vecs1 = []

for qu1 in tqdm(list(df['cleanQ1'])):
    
    doc1 = nlp(qu1) #creating object of   GLOVE model  so that we can get vetor representation of our words
    
    # Creating a matrix of N x M where N is is number of word is given line and M i.e. 96 which is the vector representaion of 1st word
    mean_vec1 = np.zeros([len(doc1), len(doc1[0].vector)])
    
    # Looping to all words in the given sentence 
    
    for word1 in doc1:
        
       # word2vec ( Creating Vector Representation of every word ) which is 96
    
        vec1 = word1.vector
        
        # Using try and catch to prefent key error [ For the words that are not there in our word2tfidf dict like empty space ]
        
        try:
            idf = word2tfidf[str(word1)]
        except:
            idf = 0
        
        # adding up all the words generated in the matrix (word2vec matrix * the word2tfidf Corresponding to that word)
        mean_vec1 += vec1 * idf
    mean_vec1 = mean_vec1.mean(axis=0)
    vecs1.append(mean_vec1)  # Storing the vector representation of every sentence into an array
df['q1_feats_m'] = list(vecs1)

100%|██████████| 161582/161582 [14:36<00:00, 184.29it/s]
<ipython-input-11-a611cba2f2aa>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['q1_feats_m'] = list(vecs1)


In [12]:
# en_vectors_web_lg, which includes over 1 million unique vectors.
nlp = spacy.load('en_core_web_sm')

vecs2 = []

for qu2 in tqdm(list(df['cleanQ2'])):
    
    doc2 = nlp(qu2) #creating object of   GLOVE model  so that we can get vetor representation of our words
    
    # Creating a matrix of N x M where N is is number of word is given line and M i.e. 96 which is the vector representaion of 1st word
    mean_vec2 = np.zeros([len(doc2), len(doc2[0].vector)])
    
    # Looping to all words in the given sentence 
    
    for word2 in doc2:
        
       # word2vec ( Creating Vector Representation of every word ) which is 96
    
        vec2 = word2.vector
        
        # Using try and catch to prefent key error [ For the words that are not there in our word2tfidf dict like empty space ]
        
        try:
            idf = word2tfidf[str(word2)]
        except:
            idf = 0
        
        # adding up all the words generated in the matrix (word2vec matrix * the word2tfidf Corresponding to that word)
        mean_vec2 += vec2 * idf
    mean_vec2 = mean_vec2.mean(axis=0)
    vecs2.append(mean_vec2)  # Storing the vector representation of every sentence into an array
df['q2_feats_m'] = list(vecs2)

100%|██████████| 161582/161582 [13:16<00:00, 202.80it/s]
<ipython-input-12-b74d79ec72aa>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['q2_feats_m'] = list(vecs2)


In [13]:
df.head()

,cleanQ1,cleanQ2,is_duplicate,q1_feats_m,q2_feats_m
0,legal ethic issu relat boyt good untri,polit divis issu untri would bizarr peopl untr...,0,"[105.28242015838623, 33.92298239469528, -14.66...","[34.391451597213745, 81.68937733769417, -110.2..."
1,remov democraci india,biggest blunder made vajpaye govern,0,"[6.738011121749878, -2.32770474255085, -32.116...","[-48.09071063995361, 25.820831298828125, -34.6..."
2,watch fox news addict,watch fox news bad,0,"[2.490405559539795, 71.28173542022705, -61.842...","[18.46064281463623, 46.16521120071411, -47.332..."
3,arnab goswami quit time,arnab goswami resign,1,"[32.49494844675064, 7.052377223968506, -74.483...","[-3.0891857147216797, -0.7452178001403809, -57..."
4,best friend date crush,best friend date crush,1,"[-17.761229991912842, 66.89253807067871, -66.7...","[-17.761229991912842, 66.89253807067871, -66.7..."


In [14]:
df1 = df_data.drop(['cleanQ1', 'cleanQ2'], axis=1)

In [15]:
df3 = df.drop(['cleanQ1', 'cleanQ2', 'is_duplicate'],axis=1)

In [16]:
df3_q1 = pd.DataFrame(df3.q1_feats_m.values.tolist(), index= df3.index)
df3_q2 = pd.DataFrame(df3.q2_feats_m.values.tolist(), index= df3.index)

In [17]:
import os
if not os.path.isfile('../data/processed/final_features.csv'):
    df3_q1['id']=df1['id']
    df3_q2['id']=df1['id']
    #df1  = df1.merge(df2, on='id',how='left')
    df2  = df3_q1.merge(df3_q2, on='id',how='left')
    result  = df1.merge(df2, on='id',how='left')
    result.to_csv('../data/processed/final_features.csv', index=False)

In [18]:
result.head()

NameError: name 'result' is not defined